**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga](https://drive.google.com/file/d/12L7rOCgW7NEKl_xJbIGZz05XXVrOaPBz/view).

In [6]:
!pip3 install --user -q cvxpy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Constrained Equality Convex Optimization (CECO)

Se comparan los resultados del **paquete [cvxpy](https://github.com/cvxgrp/cvxpy)** con los obtenidos en la implementación hecha por el prof en [algoritmos/Python](algoritmos/Python), en específico [algoritmos/Python/algorithms_for_ceco.py](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/IV.optimizacion_convexa_y_machine_learning/algoritmos/Python/algorithms_for_uco.py) para problemas tipo CECO (*Constrained Equality Convex Optimization*)

In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/algoritmos/Python'

In [4]:
os.chdir(cur_directory + dir_alg_python)

In [5]:
import math

import numpy as np

import algorithms_for_ceco
from line_search import line_search_by_backtracking
from utils import compute_error

# Primer ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad x_1^2 + 2x_1x_2 + x_2^2-2x_2$$

$$\text{sujeto a: } x_1 = 0$$

In [6]:
fo = lambda x: x[0]**2 + 2*x[0]*x[1]+x[1]**2-2*x[1]

In [7]:
A = np.array([1,0],dtype=float)

In [8]:
b = np.array([0])

In [9]:
x_ast=np.array([0,1], dtype=float)

In [10]:
x_0 = np.array([0,-2],dtype=float)

In [11]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	7.21e+00	1.80e+01	3.00e+00	9.00e+00	---		9.00e+03
1	7.21e+00	1.15e-05	2.40e-03	5.76e-06	1.00e+00	9.00e+03
2	7.21e+00	7.55e-15	6.68e-08	4.44e-15	1.00e+00	9.00e+03
Error of x with respect to x_ast: 6.68e-08
Approximate solution: [4.33680869e-19 9.99999933e-01]


In [12]:
x

array([4.33680869e-19, 9.99999933e-01])

In [13]:
total_of_iterations

3

In [14]:
x_plot.shape

(2, 3)

In [15]:
x_plot

array([[ 0.00000000e+00,  0.00000000e+00,  4.33680869e-19],
       [-2.00000000e+00,  1.00239973e+00,  9.99999933e-01]])

In [16]:
compute_error(x_ast,x)

6.682210107467057e-08

In [17]:
x_0 = np.array([1,-2],dtype=float)

In [18]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	8.01e+00	3.16e+00	6.00e+00	---		2.03e+07
1	4.47e+00	5.12e-06	1.41e+00	2.00e+00	1.00e+00	2.03e+07
2	4.47e+00	9.86e-16	1.41e+00	2.00e+00	1.00e+00	2.03e+07
Error of x with respect to x_ast: 1.41e+00
Approximate solution: [ 1.00000000e+00 -3.34455919e-08]


In [19]:
compute_error(x_ast,x)

1.4142135860226999

In [20]:
def norm_residual(feas_primal, feas_dual):
    return np.sqrt(np.linalg.norm(feas_primal)**2 +\
                   np.linalg.norm(feas_dual)**2
                   )

In [21]:
def line_search_for_residual_by_backtracking(r_primal, r_dual,dir_desc_primal,dir_desc_dual,x, nu,
                                             norm_residual_eval,
                                             alpha=.15, beta=.5):
    """
    Line search that sufficiently decreases f restricted to a ray in the direction dir_desc.
    Args:
        alpha (float): parameter in line search with backtracking, tipically .15
        beta (float): parameter in line search with backtracking, tipically .5
        r (fun): definition of residual as function definition or lambda expression.
        dir_desc (array): descent direction.
        x (array): numpy array that holds values where line search will be performed.
        der_direct (float): directional derivative of f.
    Returns:
        t (float): positive number for stepsize along dir_desc that sufficiently decreases f.
    """
    t=1
    if alpha > 1/2:
        print('alpha must be less than or equal to 1/2')
        t=-1
    if beta>1:
        print('beta must be less than 1')
        t=-1;   
    if t!=-1:
        feas_primal = r_primal(x + t*dir_desc_primal)
        feas_dual = r_dual(nu + t*dir_desc_dual )
        eval1 = norm_residual(feas_primal, feas_dual)
        eval2 = (1-alpha*t)*norm_residual_eval
        while eval1 > eval2:
            t=beta*t
            feas_primal = r_primal(x + t*dir_desc_primal)
            feas_dual = r_dual(nu + t*dir_desc_dual )
            eval1 = norm_residual(feas_primal, feas_dual)
            eval2 = (1-alpha*t)*norm_residual_eval
    return t

In [22]:
from numerical_differentiation import gradient_approximation, \
                                      Hessian_approximation

In [23]:
def Newtons_method_infeasible_init_point(f, A, b, x_0, nu_0, tol, 
                                         tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                                         gf_symbolic = None,
                                         Hf_symbolic = None):
    '''
    Newton's method to numerically approximate solution of min f subject to Ax = b.
    Args:
        f (fun): definition of function f as lambda expression or function definition.
        A (numpy ndarray): 2d numpy array of shape (m,n) defines system of constraints Ax=b.
        x_0 (numpy ndarray): initial point for Newton's method. 
        nu_0 (numpy ndarray): initial point for Newton's method.
        tol (float): tolerance that will halt method. Controls stopping criteria.
        tol_backtracking (float): tolerance that will halt method. Controls value of line search by backtracking.
        x_ast (numpy ndarray): solution of min f, now it's required that user knows the solution...
        p_ast (float): value of f(x_ast), now it's required that user knows the solution...
        maxiter (int): maximum number of iterations
        gf_symbolic (fun): definition of gradient of f. If given, no approximation is
                                     performed via finite differences.
        Hf_symbolic (fun): definition of Hessian of f. If given, no approximation is
                                     performed via finite differences.
    Returns:
        x (numpy ndarray): numpy array, approximation of x_ast.
        iteration (int): number of iterations.
        Err_plot (numpy ndarray): numpy array of absolute error between p_ast and f(x) with x approximation
                          of x_ast. Useful for plotting.
        x_plot (numpy ndarray): numpy array that containts in columns vector of approximations. Last column
                        contains x, approximation of solution. Useful for plotting.
    '''
    iteration = 0
    x = x_0
    nu = nu_0
    
    feval = f(x)
    
    if gf_symbolic:
        gfeval = gf_symbolic(x)
    else:
        gfeval = gradient_approximation(f,x)

    if Hf_symbolic:
        Hfeval = Hf_symbolic(x)
    else:
        Hfeval = Hessian_approximation(f,x)
    
    normgf = np.linalg.norm(gfeval)
    condHf= np.linalg.cond(Hfeval)
    
    Err_plot_aux = np.zeros(maxiter)
    Err_plot_aux[iteration]=compute_error(p_ast,feval)
    
    Err = compute_error(x_ast,x)
    
        
    if(A.ndim == 1):
        p = 1
        n = x.size
        zero_matrix = np.zeros(p)
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
    else:
        p,n = A.shape
        zero_matrix = np.zeros((p,p))
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.column_stack((A,zero_matrix))
        
    x_plot = np.zeros((n,maxiter))
    x_plot[:,iteration] = x
    
    system_matrix = np.row_stack((first_stack,second_stack))
    
    residual_primal = lambda x_fun: A@x_fun-b
    
    def residual_dual(nu_fun):
        if(A.ndim==1):
            return gfeval + A.T*nu_fun
        else:
            return gfeval + A.T@nu_fun
    
    feasibility_primal = residual_primal(x)
    feasibility_dual = residual_dual(nu)
    
    rhs = np.row_stack((feasibility_dual.reshape(n,1), feasibility_primal.reshape(p,1))).T[0]

    #Newton's direction and Newton's decrement
    dir_desc = np.linalg.solve(system_matrix, -rhs)
    dir_Newton_primal = dir_desc[0:n]
    dec_Newton = -gfeval.dot(dir_Newton_primal)
    dir_Newton_dual = dir_desc[n:(n+p)]

    norm_residual_eval = norm_residual(feasibility_primal,
                                       feasibility_dual)

    norm_residual_primal = np.linalg.norm(feasibility_primal)
    norm_residual_dual = np.linalg.norm(feasibility_dual)
    print('I\t||res_primal||\t||res_dual|| \tNewton Decrement\tError x_ast\tError p_ast\tline search\tCondHf')
    print('{}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{}\t\t{:0.2e}'.format(iteration,norm_residual_primal,
                                                                                  norm_residual_dual,
                                                                                  dec_Newton,Err,
                                                                                  Err_plot_aux[iteration],"---",
                                                                                  condHf))
    
    #stopping_criteria = np.linalg.norm(feasibility_primal) > tol and \
    #                    norm_residual_eval > tol
    stopping_criteria = norm_residual_eval > tol
    iteration+=1
    while(stopping_criteria>tol and iteration < maxiter):
        der_direct = -dec_Newton
        t = line_search_for_residual_by_backtracking(residual_primal, residual_dual,
                                                     dir_Newton_primal, dir_Newton_dual,
                                                     x, nu,
                                                     norm_residual_eval
                                                     )
        x = x + t*dir_Newton_primal
        nu = nu + t*dir_Newton_dual
        feval = f(x)
        
        
        if gf_symbolic:
            gfeval = gf_symbolic(x)
        else:
            gfeval = gradient_approximation(f,x)
        
        if Hf_symbolic:
            Hfeval = Hf_symbolic(x)
        else:
            Hfeval = Hessian_approximation(f,x)
        if(A.ndim == 1):
            p = 1
            n = x.size
            zero_matrix = np.zeros(p)
            first_stack = np.column_stack((Hfeval, A.T))
            second_stack = np.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
        else:
            p,n = A.shape
            zero_matrix = np.zeros((p,p))
            first_stack = np.column_stack((Hfeval, A.T))
            second_stack = np.column_stack((A,zero_matrix))

        system_matrix = np.row_stack((first_stack,second_stack))
        
        feasibility_primal = residual_primal(x)
        feasibility_dual = residual_dual(nu)
        rhs = np.row_stack((feasibility_dual.reshape(n,1), feasibility_primal.reshape(p,1))).T[0]
            
        #Newton's direction and Newton's decrement
        dir_desc = np.linalg.solve(system_matrix, -rhs)
        dir_Newton_primal = dir_desc[0:n]
        dec_Newton = -gfeval.dot(dir_Newton_primal)
        dir_Newton_dual = dir_desc[n:(n+p)]
        
        Err_plot_aux[iteration]=compute_error(p_ast,feval)
        x_plot[:,iteration] = x
        Err = compute_error(x_ast,x)
        norm_residual_eval = norm_residual(feasibility_primal,
                                           feasibility_dual)
        norm_residual_primal = np.linalg.norm(feasibility_primal)
        norm_residual_dual = np.linalg.norm(feasibility_dual)
        print('{}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}\t{:0.2e}'.format(iteration,norm_residual_primal,
                                                                                         norm_residual_dual,
                                                                                         dec_Newton,Err,
                                                                                         Err_plot_aux[iteration],t,
                                                                                         condHf))
        #stopping_criteria = np.linalg.norm(feasibility_primal) > tol and \
        #                    norm_residual_eval > tol
        stopping_criteria = norm_residual_eval > tol
        if t<tol_backtracking: #if t is less than tol_backtracking then we need to check the reason
            iter_salida=iteration
            iteration = maxiter - 1
        iteration+=1
    print('{} {:0.2e}'.format("Error of x with respect to x_ast:",Err))
    print('{} {}'.format("Approximate solution:", x))
    cond = Err_plot_aux > np.finfo(float).eps*10**(-2)
    Err_plot = Err_plot_aux[cond]
    
    if iteration == maxiter and t < tol_backtracking:
        print("Backtracking value less than tol_backtracking, check approximation")
        iteration=iter_salida
        x_plot = x_plot[:,:iteration]
    else:
        x_plot = x_plot[:,:iteration]
    return [x,iteration,Err_plot,x_plot]

In [24]:
x_0 = np.array([1,-2], dtype=float)
nu_0 = 5

In [25]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)


In [26]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	5.00e+00	1.00e+01	3.16e+00	6.00e+00	---		2.03e+07
1	5.00e-01	2.50e+00	3.00e+00	1.58e+00	2.00e+00	5.00e-01	2.03e+07
2	2.50e-01	1.25e+00	9.99e-01	7.90e-01	7.49e-01	5.00e-01	2.03e+07
3	1.25e-01	6.25e-01	3.75e-01	3.95e-01	3.12e-01	5.00e-01	2.03e+07
4	6.25e-02	3.12e-01	1.56e-01	1.98e-01	1.41e-01	5.00e-01	2.03e+07
5	3.13e-02	1.56e-01	7.03e-02	9.88e-02	6.64e-02	5.00e-01	2.03e+07
6	1.56e-02	7.81e-02	3.32e-02	4.94e-02	3.22e-02	5.00e-01	2.03e+07
7	7.81e-03	3.90e-02	1.61e-02	2.47e-02	1.59e-02	5.00e-01	2.03e+07
8	3.91e-03	1.95e-02	7.93e-03	1.23e-02	7.87e-03	5.00e-01	2.03e+07
9	1.95e-03	9.76e-03	3.94e-03	6.17e-03	3.92e-03	5.00e-01	2.03e+07
10	9.77e-04	4.88e-03	1.96e-03	3.09e-03	1.96e-03	5.00e-01	2.03e+07
11	4.88e-04	2.44e-03	9.78e-04	1.54e-03	9.78e-04	5.00e-01	2.03e+07
12	2.44e-04	1.22e-03	4.89e-04	7.72e-04	4.89e-04	5.00e-01	2.03e+07
13	1.22e-04	6.10e-04	2.44e-04	3.86e-04	2.44e-04	5.00e-01	2.03e+07

In [27]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	5.00e+00	1.00e+01	3.16e+00	6.00e+00	---		2.03e+07
1	5.00e-01	2.50e+00	3.00e+00	1.58e+00	2.00e+00	5.00e-01	2.03e+07
2	2.50e-01	1.25e+00	9.99e-01	7.90e-01	7.49e-01	5.00e-01	2.03e+07
3	1.25e-01	6.25e-01	3.75e-01	3.95e-01	3.12e-01	5.00e-01	2.03e+07
4	6.25e-02	3.12e-01	1.56e-01	1.98e-01	1.41e-01	5.00e-01	2.03e+07
5	3.13e-02	1.56e-01	7.03e-02	9.88e-02	6.64e-02	5.00e-01	2.03e+07
6	1.56e-02	7.81e-02	3.32e-02	4.94e-02	3.22e-02	5.00e-01	2.03e+07
7	7.81e-03	3.90e-02	1.61e-02	2.47e-02	1.59e-02	5.00e-01	2.03e+07
8	3.91e-03	1.95e-02	7.93e-03	1.23e-02	7.87e-03	5.00e-01	2.03e+07
9	1.95e-03	9.76e-03	3.94e-03	6.17e-03	3.92e-03	5.00e-01	2.03e+07
10	9.77e-04	4.88e-03	1.96e-03	3.09e-03	1.96e-03	5.00e-01	2.03e+07
11	4.88e-04	2.44e-03	9.78e-04	1.54e-03	9.78e-04	5.00e-01	2.03e+07
12	2.44e-04	1.22e-03	4.89e-04	7.72e-04	4.89e-04	5.00e-01	2.03e+07
13	1.22e-04	6.10e-04	2.44e-04	3.86e-04	2.44e-04	5.00e-01	2.03e+07

In [28]:
compute_error(x_ast,x)

1.1430297930381528e-08

# Segundo ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad \frac{1}{2}(x_1^2 + x_2^2)$$

$$\text{sujeto a: } 2x_1 -x_2 = 5$$

In [29]:
fo = lambda x: 1/2*(x[0]**2  + x[1]**2)

In [30]:
A = np.array([2,-1],dtype=float)

In [31]:
b=5

In [32]:
x_ast = np.array([2,-1],dtype=float)

In [33]:
x_0 = np.array([0,-5],dtype=float)

In [34]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	5.00e+00	2.00e+01	2.00e+00	4.00e+00	---		1.00e+00
1	5.00e+00	3.94e-06	8.88e-04	7.89e-07	1.00e+00	1.00e+00
2	5.00e+00	3.56e-12	8.63e-07	7.45e-13	1.00e+00	1.00e+00
Error of x with respect to x_ast: 8.63e-07
Approximate solution: [ 1.99999914 -1.00000173]


In [35]:
x

array([ 1.99999914, -1.00000173])

In [36]:
x_ast

array([ 2., -1.])

In [37]:
compute_error(x_ast,x)

8.631750624932266e-07

In [38]:
x_0 = np.array([1,1],dtype=float)
nu_0 = 4

In [39]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.00e+00	9.49e+00	1.00e+00	1.00e+00	6.00e-01	---		1.00e+00
1	0.00e+00	7.56e-04	3.19e-08	7.99e-05	6.38e-09	1.00e+00	1.00e+00
2	0.00e+00	1.12e-07	9.87e-15	4.08e-08	1.60e-15	1.00e+00	1.00e+00
3	0.00e+00	5.99e-08	3.55e-15	1.86e-08	3.55e-16	5.00e-01	1.00e+00
4	0.00e+00	2.58e-08	-0.00e+00	1.86e-08	3.55e-16	2.22e-16	1.00e+00
Error of x with respect to x_ast: 1.86e-08
Approximate solution: [ 1.99999998 -1.00000004]
Backtracking value less than tol_backtracking, check approximation


In [40]:
compute_error(x_ast,x)

1.8573515481179468e-08

# Tercer ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad x_1^2 + x_2^2$$

$$\text{sujeto a :} x_1+x_2 = 1$$

In [41]:
fo = lambda x: x[0]**2+x[1]**2

In [42]:
x_ast = np.array([.5,.5],dtype=float)

In [43]:
A = np.array([1,1],dtype=float)

In [44]:
b=1

In [45]:
x_0 = np.array([2,-1],dtype=float)

In [46]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	9.00e+00	3.00e+00	9.00e+00	---		1.00e+00
1	4.47e+00	7.11e-08	2.67e-04	7.11e-08	1.00e+00	1.00e+00
2	4.47e+00	1.81e-16	2.46e-12	1.11e-16	1.00e+00	1.00e+00
Error of x with respect to x_ast: 2.46e-12
Approximate solution: [0.5 0.5]


In [47]:
compute_error(x_ast,x)

2.4647506264441902e-12

In [48]:
x_0 = np.array([1,1],dtype=float)
nu_0 = 4

In [49]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	8.49e+00	2.00e+00	1.00e+00	3.00e+00	---		1.00e+00
1	1.11e-15	1.26e-04	-1.08e-15	1.12e-15	2.44e-15	1.00e+00	1.00e+00
2	0.00e+00	7.85e-09	-8.27e-25	5.55e-09	0.00e+00	1.00e+00	1.00e+00
Error of x with respect to x_ast: 5.55e-09
Approximate solution: [0.5 0.5]


In [50]:
compute_error(x_ast,x)

5.550929882414124e-09

# Cuarto ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad e^{x_1+3x_2-0.1} + e^{x_1 -3x_2-0.1} + e^{-x_1-0.1}$$

$$\text{sujeto a:} x_1 + 3x_2 = 0$$

In [51]:
fo = lambda x: math.exp(x[0] + 3*x[1]-0.1) + math.exp(x[0]  -3*x[1]-0.1) + math.exp(-x[0]-0.1)

In [52]:
x_ast = np.array([-0.23104907880100917,0.0770163596518852],dtype=float)

In [53]:
x_0 = np.array([0,0],dtype=float)

In [54]:
A = np.array([1,3],dtype=float)

In [55]:
b=0

In [56]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	9.05e-01	1.81e-01	1.00e+00	3.81e-02	---		6.00e+00
1	9.05e-01	3.30e-03	1.34e-01	6.37e-04	1.00e+00	6.00e+00
2	9.05e-01	8.46e-07	2.15e-03	1.62e-07	1.00e+00	6.00e+00
3	9.05e-01	7.41e-14	6.52e-07	1.98e-11	1.00e+00	6.00e+00
Error of x with respect to x_ast: 6.52e-07
Approximate solution: [-0.23104893  0.07701631]


In [57]:
compute_error(x_ast, x)

6.524588707146708e-07

In [58]:
x_0 = np.array([1,1],dtype=float) #with 1,1 doesn't work
nu_0 = 1

In [59]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.00e+00	1.59e+02	1.97e+02	6.32e+00	1.81e+01	---		6.62e+02
1	4.00e+00	1.59e+02	1.97e+02	6.32e+00	1.81e+01	2.78e-17	6.62e+02
Error of x with respect to x_ast: 6.32e+00
Approximate solution: [1. 1.]
Backtracking value less than tol_backtracking, check approximation


In [60]:
compute_error(x_ast,x)

6.317584489434056

In [61]:
x_0 = np.array([0,0],dtype=float) 
nu_0 = 1

In [62]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	3.55e+00	1.81e-01	1.00e+00	3.81e-02	---		6.00e+00
1	1.11e-16	1.96e-01	3.30e-03	1.34e-01	6.37e-04	1.00e+00	6.00e+00
2	0.00e+00	3.38e-03	8.33e-07	2.14e-03	1.59e-07	1.00e+00	6.00e+00
3	0.00e+00	1.06e-06	6.98e-14	7.34e-07	1.98e-11	1.00e+00	6.00e+00
4	0.00e+00	5.92e-08	1.03e-15	1.16e-07	1.98e-11	1.00e+00	6.00e+00
5	2.78e-17	1.78e-08	5.48e-17	9.73e-08	1.98e-11	2.50e-01	6.00e+00
6	2.78e-17	1.78e-08	5.48e-17	9.73e-08	1.98e-11	2.22e-16	6.00e+00
Error of x with respect to x_ast: 9.73e-08
Approximate solution: [-0.23104906  0.07701635]
Backtracking value less than tol_backtracking, check approximation


In [63]:
compute_error(x_ast,x)

9.732233305891635e-08

# Quinto ejemplo: con más restricciones de igualdad

Resolver: 
$$\displaystyle \min_{x \in \mathbb{R}^3} \quad ||x||_2^2$$

$$\text{sujeto a: }\begin{array}{l}
\begin{array}{c}
x_1 + x_2 + x_3 = 1 \\
x_1 + x_2 + 2x_3 = 3
\end{array}
\end{array}
$$

In [64]:
fo = lambda x: x.dot(x)

In [65]:
x_ast = np.array([-0.5,-0.5,2. ], dtype=float)

In [66]:
A = np.array([[1, 1, 1],
              [1, 1, 2]],dtype=float)

In [67]:
b = np.array([1,3])

In [68]:
x_0 = np.array([3,-4,2], dtype=float)
#x_0 = np.array([2,-3,2], dtype=float)

In [69]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)


In [70]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.08e+01	4.90e+01	2.33e+00	5.44e+00	---		1.00e+00
1	1.08e+01	3.14e-05	1.87e-03	3.48e-06	1.00e+00	1.00e+00
2	1.08e+01	5.83e-13	2.52e-07	6.45e-14	1.00e+00	1.00e+00
Error of x with respect to x_ast: 2.52e-07
Approximate solution: [-0.49999962 -0.50000038  2.        ]


In [71]:
compute_error(x_ast,x)

2.518419818612709e-07

## Punto infactible

In [72]:
x_0 = np.array([3,-4,2], dtype=float) #Ax_0 = b
#x_0 = np.array([2,-3,2], dtype=float)
nu_0 = np.array([0,0],dtype=float)

In [73]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	1.08e+01	4.90e+01	2.33e+00	5.44e+00	---		1.00e+00
1	6.28e-16	1.01e+01	4.30e+01	2.19e+00	4.78e+00	6.25e-02	1.00e+00
2	1.26e-15	9.47e+00	3.79e+01	2.05e+00	4.21e+00	6.25e-02	1.00e+00
3	0.00e+00	8.87e+00	3.33e+01	1.92e+00	3.70e+00	6.25e-02	1.00e+00
4	6.28e-16	8.32e+00	2.92e+01	1.80e+00	3.25e+00	6.25e-02	1.00e+00
5	1.26e-15	7.80e+00	2.57e+01	1.69e+00	2.85e+00	6.25e-02	1.00e+00
6	6.28e-16	7.31e+00	2.26e+01	1.58e+00	2.51e+00	6.25e-02	1.00e+00
7	6.28e-16	6.85e+00	1.99e+01	1.49e+00	2.21e+00	6.25e-02	1.00e+00
8	6.28e-16	6.43e+00	1.74e+01	1.39e+00	1.94e+00	6.25e-02	1.00e+00
9	6.28e-16	6.02e+00	1.53e+01	1.31e+00	1.70e+00	6.25e-02	1.00e+00
10	1.26e-15	5.65e+00	1.35e+01	1.22e+00	1.50e+00	6.25e-02	1.00e+00
11	1.26e-15	5.30e+00	1.18e+01	1.15e+00	1.32e+00	6.25e-02	1.00e+00
12	6.28e-16	4.96e+00	1.04e+01	1.08e+00	1.16e+00	6.25e-02	1.00e+00
13	6.28e-16	4.65e+00	9.16e+00	1.01e+00	1.02e+00	6.25e-02	1.00e+00

In [74]:
compute_error(x_ast,x)

0.09875671809487886

In [75]:
x_0 = np.array([3,-4,0], dtype=float) #Ax_0 different to b
#x_0 = np.array([2,-3,2], dtype=float)
nu_0 = np.array([0,0],dtype=float)

In [76]:
[x,total_of_iterations,
 Err_plot,x_plot]=Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter)

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	1.00e+01	4.90e+01	2.52e+00	4.56e+00	---		1.00e+00
1	3.91e+00	8.75e+00	3.66e+01	2.20e+00	3.29e+00	1.25e-01	1.00e+00
2	3.42e+00	7.66e+00	2.73e+01	1.93e+00	2.35e+00	1.25e-01	1.00e+00
3	3.00e+00	6.70e+00	2.02e+01	1.69e+00	1.65e+00	1.25e-01	1.00e+00
4	2.62e+00	5.86e+00	1.49e+01	1.48e+00	1.13e+00	1.25e-01	1.00e+00
5	2.29e+00	5.13e+00	1.09e+01	1.29e+00	7.55e-01	1.25e-01	1.00e+00
6	2.01e+00	4.49e+00	7.89e+00	1.13e+00	4.78e-01	1.25e-01	1.00e+00
7	1.76e+00	3.93e+00	5.65e+00	9.88e-01	2.79e-01	1.25e-01	1.00e+00
8	1.54e+00	3.44e+00	3.98e+00	8.65e-01	1.37e-01	1.25e-01	1.00e+00
9	1.34e+00	3.01e+00	2.75e+00	7.57e-01	3.82e-02	1.25e-01	1.00e+00
10	1.18e+00	2.63e+00	1.84e+00	6.62e-01	2.92e-02	1.25e-01	1.00e+00
11	1.03e+00	2.30e+00	1.18e+00	5.79e-01	7.35e-02	1.25e-01	1.00e+00
12	9.01e-01	2.01e+00	7.02e-01	5.07e-01	1.01e-01	1.25e-01	1.00e+00
13	7.88e-01	1.76e+00	3.61e-01	4.44e-01	1.17e-01	1.25e-01	1.00e+00

In [77]:
compute_error(x_ast,x)

0.003625150358167456

# Comparación del quinto ejemplo con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [212]:
import cvxpy as cp

In [52]:
x = cp.Variable(3)
A = np.array([[1, 1, 1],
              [1, 1, 2]])
b = np.array([1,3])

In [55]:
obj = cp.Minimize(cp.norm(x,2))

constraints = [A@x == b]

In [56]:
prob = cp.Problem(obj, constraints)

In [57]:
prob.solve()

2.1213203277662585

In [58]:
print("optimal value", prob.value)


optimal value 2.1213203277662585


In [59]:
print("optimal var", x.value)

optimal var [-0.5 -0.5  2. ]


In [60]:
A@x.value

array([1., 3.])

In [61]:
np.linalg.norm(x.value)

2.121320343558957

## Otros puntos iniciales

In [62]:
vec = [1,-2,2]
A@vec

array([1, 3])

In [63]:
np.linalg.norm(vec)

3.0

In [64]:
vec = [2,-3,2]
A@vec

array([1, 3])

In [65]:
np.linalg.norm(vec)

4.123105625617661

In [66]:
vec = [3,-4,2]
A@vec

array([1, 3])

In [67]:
np.linalg.norm(vec)

5.385164807134504

**Referencias:**

* S. P. Boyd, L. Vandenberghe, Convex Optimization, Cambridge University Press, 2009.